In [1]:
import numpy as np
import pandas as pd
import re
from gensim.models import Word2Vec
from keras.preprocessing.text import Tokenizer
from keras.utils.data_utils import pad_sequences
from sklearn.model_selection import train_test_split
from keras.layers import Embedding, Conv1D, MaxPooling1D, Dense, Flatten, LSTM, Bidirectional
from keras.models import Sequential
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# 選擇數據處理模式：'N-R' 或 'R'
mode = 'N-R'  # 可以選擇 'N-R' 或 'R'

# 載入與預處理數據
def load_semeval_data(file_paths):
    data = []
    labels = []
    for file_path in file_paths:
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                parts = line.strip().split('\t')
                if len(parts) == 3:
                    data.append(parts[2])
                    labels.append(parts[1])
    return pd.DataFrame({'tweet': data, 'label': labels})

file_paths = [
    './dataset/train/twitter-2013train-A.txt',
    './dataset/train/twitter-2015train-A.txt',
    './dataset/train/twitter-2016train-A.txt',
]

dataset = load_semeval_data(file_paths)

# 預處理推文文本
def preprocess_tweet(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r"https?:\/\/\S+", "", tweet)
    tweet = re.sub(r"[^a-zA-Z0-9\s]", "", tweet)
    return tweet

dataset['tweet'] = dataset['tweet'].apply(preprocess_tweet)

# Tokenization 和 Padding 設置
MAX_NB_WORDS = 10000
MAX_SEQUENCE_LENGTH = 40  # 用於 N-R 模式
REGION_SIZE = 10  # 每個區域的長度，用於 R 模式
NUM_REGIONS = 4  # 區域數量，用於 R 模式
EMBEDDING_DIM = 25

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(dataset['tweet'].values)
word_index = tokenizer.word_index

# 根據模式選擇不同的數據處理方法
if mode == 'N-R':
    # Non-Regional 模式，直接將推文轉為固定長度序列
    X = tokenizer.texts_to_sequences(dataset['tweet'].values)
    X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
    input_length = MAX_SEQUENCE_LENGTH  # 設置模型的 input_length
elif mode == 'R':
    # Regional 模式，將推文分成多個區域並填充至相同長度
    X = tokenizer.texts_to_sequences(dataset['tweet'].values)
    X_padded = []
    for tweet in X:
        tweet_regions = []
        for i in range(NUM_REGIONS):
            start = i * REGION_SIZE
            end = start + REGION_SIZE
            region = tweet[start:end]
            if len(region) < REGION_SIZE:
                region = region + [0] * (REGION_SIZE - len(region))
            tweet_regions.extend(region)
        X_padded.append(tweet_regions)
    X = np.array(X_padded)
    input_length = NUM_REGIONS * REGION_SIZE  # 設置模型的 input_length
else:
    raise ValueError("Invalid mode. Choose 'N-R' or 'R'.")

# 標籤處理
label_mapping = {'positive': 0, 'negative': 1, 'neutral': 2}
y = dataset['label'].map(label_mapping).values
y = pd.get_dummies(y).values

# 數據分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 訓練 Word2Vec 模型
tokenized_sentences = [tweet.split() for tweet in dataset['tweet'].values]
word2vec_model = Word2Vec(sentences=tokenized_sentences,
                          vector_size=EMBEDDING_DIM,  # 詞向量維度為 25
                          window=5,                   # 最大跳躍距離設為 5
                          min_count=5,                # 最低頻次為 5
                          sg=0,                       # CBOW 模型 (sg=0)
                          workers=4)                  # 使用 4 個核心進行訓練

# 將詞向量進行標準化
def normalize_vector(vector):
    v_min, v_max = np.min(vector), np.max(vector)
    return (vector - v_min) / (v_max - v_min) if v_max > v_min else vector

# 構建嵌入矩陣
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    if word in word2vec_model.wv:
        embedding_vector = word2vec_model.wv[word]
        embedding_matrix[i] = normalize_vector(embedding_vector)

# 定義嵌入層，用於後續模型
embedding_layer = Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
                            input_length=input_length, trainable=False)

### 建立單一 CNN 模型並加入 Flatten 層

In [ ]:
from keras.layers import Flatten

cnn_model = Sequential()
cnn_model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
                        input_length=MAX_SEQUENCE_LENGTH, trainable=False))
cnn_model.add(Conv1D(filters=12, kernel_size=3, activation='relu'))
cnn_model.add(MaxPooling1D(pool_size=3))
cnn_model.add(Flatten())  # 加入 Flatten 層
cnn_model.add(Dense(3, activation='softmax'))
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
cnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = cnn_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
405/405 [==============================] - 2s 3ms/step - loss: 1.0127 - accuracy: 0.4470 - val_loss: 0.9887 - val_accuracy: 0.4726
Epoch 2/10
405/405 [==============================] - 1s 2ms/step - loss: 0.9924 - accuracy: 0.4719 - val_loss: 0.9805 - val_accuracy: 0.4720
Epoch 3/10
405/405 [==============================] - 1s 2ms/step - loss: 0.9842 - accuracy: 0.4814 - val_loss: 0.9789 - val_accuracy: 0.4754
Epoch 4/10
405/405 [==============================] - 1s 2ms/step - loss: 0.9794 - accuracy: 0.4848 - val_loss: 0.9829 - val_accuracy: 0.4692
Epoch 5/10
405/405 [==============================] - 1s 2ms/step - loss: 0.9748 - accuracy: 0.4984 - val_loss: 0.9724 - val_accuracy: 0.4921
Epoch 6/10
405/405 [==============================] - 1s 2ms/step - loss: 0.9731 - accuracy: 0.4975 - val_loss: 0.9660 - val_accuracy: 0.4869
Epoch 7/10
405/405 [==============================] - 1s 2ms/step - loss: 0.9710 - accuracy: 0.4946 - val_loss: 0.9700 - val_accuracy: 0.4853
Epoch 

### 單一 LSTM 模型 (Single LSTM)

In [ ]:
from keras.layers import LSTM

# 建立單一 LSTM 模型
lstm_model = Sequential()
lstm_model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
                         input_length=MAX_SEQUENCE_LENGTH, trainable=False))
lstm_model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
lstm_model.add(Dense(3, activation='softmax'))
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = lstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
405/405 [==============================] - 25s 58ms/step - loss: 0.9983 - accuracy: 0.4617 - val_loss: 0.9787 - val_accuracy: 0.4501
Epoch 2/10
405/405 [==============================] - 23s 57ms/step - loss: 0.9851 - accuracy: 0.4833 - val_loss: 0.9680 - val_accuracy: 0.4822
Epoch 3/10
405/405 [==============================] - 23s 58ms/step - loss: 0.9828 - accuracy: 0.4817 - val_loss: 0.9630 - val_accuracy: 0.5005
Epoch 4/10
405/405 [==============================] - 23s 58ms/step - loss: 0.9770 - accuracy: 0.4914 - val_loss: 0.9622 - val_accuracy: 0.4943
Epoch 5/10
405/405 [==============================] - 25s 61ms/step - loss: 0.9774 - accuracy: 0.4926 - val_loss: 0.9636 - val_accuracy: 0.4785
Epoch 6/10
405/405 [==============================] - 26s 65ms/step - loss: 0.9755 - accuracy: 0.4938 - val_loss: 0.9587 - val_accuracy: 0.4924
Epoch 7/10
405/405 [==============================] - 27s 66ms/step - loss: 0.9714 - accuracy: 0.4944 - val_loss: 0.9659 - val_accuracy:

### CNN + LSTM 組合模型 (CNN + LSTM Combined Network)

In [ ]:
# 建立 CNN + LSTM 模型
cnn_lstm_model = Sequential()
cnn_lstm_model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
                             input_length=MAX_SEQUENCE_LENGTH, trainable=False))
cnn_lstm_model.add(Conv1D(filters=12, kernel_size=3, activation='relu'))
cnn_lstm_model.add(MaxPooling1D(pool_size=3))
cnn_lstm_model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
cnn_lstm_model.add(Dense(3, activation='softmax'))
cnn_lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
cnn_lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = cnn_lstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
405/405 [==============================] - 12s 23ms/step - loss: 1.0085 - accuracy: 0.4455 - val_loss: 0.9915 - val_accuracy: 0.4563
Epoch 2/10
405/405 [==============================] - 9s 22ms/step - loss: 0.9955 - accuracy: 0.4651 - val_loss: 0.9891 - val_accuracy: 0.4600
Epoch 3/10
405/405 [==============================] - 9s 22ms/step - loss: 0.9895 - accuracy: 0.4721 - val_loss: 0.9758 - val_accuracy: 0.4705
Epoch 4/10
405/405 [==============================] - 9s 21ms/step - loss: 0.9824 - accuracy: 0.4838 - val_loss: 0.9729 - val_accuracy: 0.4832
Epoch 5/10
405/405 [==============================] - 9s 21ms/step - loss: 0.9810 - accuracy: 0.4831 - val_loss: 0.9696 - val_accuracy: 0.4940
Epoch 6/10
405/405 [==============================] - 9s 21ms/step - loss: 0.9774 - accuracy: 0.4881 - val_loss: 0.9703 - val_accuracy: 0.4804
Epoch 7/10
405/405 [==============================] - 9s 21ms/step - loss: 0.9739 - accuracy: 0.4937 - val_loss: 0.9633 - val_accuracy: 0.496

c:\Users\jk121\.conda\envs\DL-113\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\jk121\.conda\envs\DL-113\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\jk121\.conda\envs\DL-113\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### CNN + LSTM 組合模型 + SVM 組合模型

In [5]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# CNN 特徵提取模型
cnn_feature_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=12, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=3),
    Flatten()
])

# LSTM 特徵提取模型
lstm_feature_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    Flatten()
])

# 提取 CNN 和 LSTM 特徵
cnn_features = cnn_feature_model.predict(X_train)
lstm_features = lstm_feature_model.predict(X_train)
combined_features = np.hstack((cnn_features, lstm_features))

# 使用 SVM 進行分類
svm_classifier = SVC(kernel='linear', probability=True)
svm_classifier.fit(combined_features, np.argmax(y_train, axis=1))

# 評估
cnn_test_features = cnn_feature_model.predict(X_test)
lstm_test_features = lstm_feature_model.predict(X_test)
combined_test_features = np.hstack((cnn_test_features, lstm_test_features))
y_pred = svm_classifier.predict(combined_test_features)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred))

102/102 [==============================] - 1s 14ms/step
              precision    recall  f1-score   support

           0       0.49      0.59      0.53      1372
           1       1.00      0.00      0.00       466
           2       0.52      0.60      0.56      1397

    accuracy                           0.51      3235
   macro avg       0.67      0.39      0.37      3235
weighted avg       0.58      0.51      0.47      3235



### 單一 3 層 CNN 和 LSTM 網絡

In [ ]:
three_layer_cnn_lstm_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=32, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    Dense(3, activation='softmax')
])

three_layer_cnn_lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
three_layer_cnn_lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = three_layer_cnn_lstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
405/405 [==============================] - 7s 12ms/step - loss: 1.0044 - accuracy: 0.4550 - val_loss: 1.0108 - val_accuracy: 0.4448
Epoch 2/10
405/405 [==============================] - 4s 11ms/step - loss: 0.9858 - accuracy: 0.4779 - val_loss: 1.0108 - val_accuracy: 0.4451
Epoch 3/10
405/405 [==============================] - 4s 11ms/step - loss: 0.9765 - accuracy: 0.4846 - val_loss: 0.9708 - val_accuracy: 0.4813
Epoch 4/10
405/405 [==============================] - 4s 11ms/step - loss: 0.9684 - accuracy: 0.4945 - val_loss: 0.9574 - val_accuracy: 0.4986
Epoch 5/10
405/405 [==============================] - 4s 11ms/step - loss: 0.9652 - accuracy: 0.4982 - val_loss: 0.9709 - val_accuracy: 0.4807
Epoch 6/10
405/405 [==============================] - 4s 11ms/step - loss: 0.9577 - accuracy: 0.5041 - val_loss: 0.9606 - val_accuracy: 0.4971
Epoch 7/10
405/405 [==============================] - 4s 11ms/step - loss: 0.9550 - accuracy: 0.5068 - val_loss: 0.9559 - val_accuracy: 0.5051

### 多重 CNN 和 LSTM 網絡

In [ ]:
multi_cnn_lstm_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    Dense(3, activation='softmax')
])

multi_cnn_lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
multi_cnn_lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = multi_cnn_lstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
405/405 [==============================] - 11s 20ms/step - loss: 1.0038 - accuracy: 0.4510 - val_loss: 0.9793 - val_accuracy: 0.4748
Epoch 2/10
405/405 [==============================] - 8s 20ms/step - loss: 0.9859 - accuracy: 0.4791 - val_loss: 0.9716 - val_accuracy: 0.4726
Epoch 3/10
405/405 [==============================] - 8s 20ms/step - loss: 0.9787 - accuracy: 0.4845 - val_loss: 0.9705 - val_accuracy: 0.4671
Epoch 4/10
405/405 [==============================] - 9s 21ms/step - loss: 0.9759 - accuracy: 0.4891 - val_loss: 0.9599 - val_accuracy: 0.4943
Epoch 5/10
405/405 [==============================] - 8s 19ms/step - loss: 0.9689 - accuracy: 0.4933 - val_loss: 0.9767 - val_accuracy: 0.4918
Epoch 6/10
405/405 [==============================] - 8s 19ms/step - loss: 0.9657 - accuracy: 0.4971 - val_loss: 0.9578 - val_accuracy: 0.5054
Epoch 7/10
405/405 [==============================] - 8s 19ms/step - loss: 0.9630 - accuracy: 0.4994 - val_loss: 0.9695 - val_accuracy: 0.479

### 單一 3 層 CNN 和雙向 LSTM 網絡

In [ ]:
from keras.layers import Bidirectional

three_layer_cnn_bilstm_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=32, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)),
    Dense(3, activation='softmax')
])

three_layer_cnn_bilstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
three_layer_cnn_bilstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = three_layer_cnn_bilstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
405/405 [==============================] - 12s 19ms/step - loss: 1.0094 - accuracy: 0.4300 - val_loss: 0.9867 - val_accuracy: 0.4427
Epoch 2/10
405/405 [==============================] - 8s 21ms/step - loss: 0.9928 - accuracy: 0.4620 - val_loss: 0.9941 - val_accuracy: 0.4699
Epoch 3/10
405/405 [==============================] - 8s 19ms/step - loss: 0.9829 - accuracy: 0.4738 - val_loss: 0.9729 - val_accuracy: 0.4807
Epoch 4/10
405/405 [==============================] - 8s 19ms/step - loss: 0.9780 - accuracy: 0.4825 - val_loss: 0.9716 - val_accuracy: 0.4714
Epoch 5/10
405/405 [==============================] - 8s 20ms/step - loss: 0.9705 - accuracy: 0.4921 - val_loss: 0.9694 - val_accuracy: 0.4844
Epoch 6/10
405/405 [==============================] - 8s 20ms/step - loss: 0.9632 - accuracy: 0.4978 - val_loss: 0.9631 - val_accuracy: 0.4989
Epoch 7/10
405/405 [==============================] - 8s 20ms/step - loss: 0.9597 - accuracy: 0.4993 - val_loss: 0.9614 - val_accuracy: 0.496

### 多重 CNN 和雙向 LSTM 網絡

In [ ]:
multi_cnn_bilstm_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)),
    Dense(3, activation='softmax')
])

multi_cnn_bilstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
multi_cnn_bilstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = multi_cnn_bilstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
405/405 [==============================] - 20s 38ms/step - loss: 1.0049 - accuracy: 0.4520 - val_loss: 0.9860 - val_accuracy: 0.4634
Epoch 2/10
405/405 [==============================] - 16s 39ms/step - loss: 0.9859 - accuracy: 0.4775 - val_loss: 0.9797 - val_accuracy: 0.4915
Epoch 3/10
405/405 [==============================] - 15s 37ms/step - loss: 0.9821 - accuracy: 0.4783 - val_loss: 0.9707 - val_accuracy: 0.4742
Epoch 4/10
405/405 [==============================] - 14s 35ms/step - loss: 0.9743 - accuracy: 0.4886 - val_loss: 0.9681 - val_accuracy: 0.4869
Epoch 5/10
405/405 [==============================] - 16s 39ms/step - loss: 0.9710 - accuracy: 0.4900 - val_loss: 0.9656 - val_accuracy: 0.4940
Epoch 6/10
405/405 [==============================] - 14s 36ms/step - loss: 0.9661 - accuracy: 0.4974 - val_loss: 0.9592 - val_accuracy: 0.4940
Epoch 7/10
405/405 [==============================] - 16s 40ms/step - loss: 0.9611 - accuracy: 0.5010 - val_loss: 0.9633 - val_accuracy: